# Neural Style Transfer

In this project we trained a custom Neural Style Transfer (NST) model that can take a realisic content image and apply a style (painting).

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optimization

import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.utils import save_image

from tqdm.notebook import tqdm
from PIL import Image

### Content loss function

We'll implement a function to calculate the content loss which is the squared error loss between the two feature vectors of the content image and the target image.

In [ ]:
def get_content_loss(target_vec, content_vec):
  return torch.mean((target_vec-content_vec)**2)

### Style loss function

We'll implement a function to calculate the style loss by using Gram matrix. The total loss is the sum of every mean-squared distance (between two gram matrices of the style and the target images) for every layer times the weighted factor (the influence factor) of every layer.

In [ ]:
def gram_matrix(input, c, h, w):
  #c-channels; h-height; w-width 
  input = input.view(c, h*w) 
  #matrix multiplication on its own transposed form
  G = torch.mm(input, input.t())
  return G
  
def get_style_loss(target, style):
  _, c, h, w = target.size()
  G = gram_matrix(target, c, h, w) #gram matrix for the target image
  S = gram_matrix(style, c, h, w) #gram matrix for the style image
  return torch.mean((G-S)**2)/(c*h*w)

### The model

In [ ]:
#class for loading the vgg19 model with wanted layers
class VGG(nn.Module):
  def __init__(self):
    super(VGG, self).__init__()
    self.select_features = ['0', '5', '10', '19', '28'] #conv layers
    self.vgg = models.vgg19(pretrained=True).features
  
  def forward(self, output):
    features = []
    for name, layer in self.vgg._modules.items():
      output = layer(output)
      if name in self.select_features:
        features.append(output)
    return features

#load the model
vgg = VGG().to(device).eval()